# Push vector data to ES

## Major

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare import data_load

doc_loader = data_load.DocDataLoader()
config = Configuration()

In [2]:
db = doc_loader.load_major_docs(size = 460, overlap=20)

From (44, 4) to Counter({2: 37, 1: 6, 3: 1})


### Gemini

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

INDEX = "gemini-major"
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=config.load_gemini_token())

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### VN SBert

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "vnsbert-major"

MODEL = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(0),
    model_name=MODEL)

### LaBSE

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "labse-major"

MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=config.load_hg_token(1),
    model_name=MODEL)

## Load

In [ ]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=config.load_elasticsearch_connection(),
                          index_name=INDEX,
                          distance_strategy="EUCLIDEAN_DISTANCE")


# Push text data to ES

## Major text 

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare.data_load import DocDataLoader

config = Configuration()
doc_loader = DocDataLoader()

db = doc_loader.load_major_docs(size=460, overlap=20)
len(db)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 4) to Counter({2: 37, 1: 6, 3: 1})


83

In [2]:
from langchain.retrievers import ElasticSearchBM25Retriever
from src.custom.es_bm25_retriever import MyElasticSearchBM25Retriever
es = config.load_elasticsearch_connection()
INDEX = "text-split-major"
retriever = MyElasticSearchBM25Retriever(client=es, index_name=INDEX)

In [3]:
a = retriever.add_documents(documents=db)

## Load

In [4]:
retriever.k = 2
a = retriever.get_relevant_documents("Hóa học")
a

[Document(page_content='Việt Nam học - Chuyên ngành Việt ngữ học và văn hóa xã hội Việt Nam là một ngành khoa học nghiên cứu về đất nước, con người Việt Nam theo hướng tiếp cận liên ngành. Chương trình đào tạo ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam tại Trường Đại học Tôn Đức Thắng là chương trình dành cho đối tượng là người nước ngoài, nhằm đáp ứng nhu cầu về nguồn nhân lực là người nước ngoài sử dụng thông thạo tiếng Việt, am hiểu về đất nước và con người Việt Nam.\n\nSau khi tốt nghiệp ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam, học viên có thể tìm kiếm việc làm phù hợp ở các cơ quan ngoại giao, tổ chức quốc tế, công ty du lịch, văn phòng thương mại, tổ chức chính phủ và phi chính phủ có nhu cầu sử dụng nguồn nhân lực am hiểu về Việt Nam (ở Việt Nam và nước ngoài). Ngoài ra sinh viên cũng có thể làm công tác phiên dịch, biên dịch, nghiên cứu và giảng dạy tiếng Việt tại các trường đại học, cao đẳng hoặc các trung tâm ngôn

In [5]:
retriever.k = 1
a = retriever.get_relevant_documents("em muốn biết ngành kỹ thuật phần mềm")
a


[Document(page_content='Sinh viên còn được chú trọng tới việc nâng cao các kỹ năng nghề nghiệp cần thiết như: rèn luyện các phương pháp vẽ minh họa, vẽ kỹ thuật trong thiết kế thời trang, hướng dẫn các kỹ thuật cắt may từ đơn giản đến phức tạp. Số tiết học thực hành được thiết kế để cân bằng với số môn học lý thuyết và mang tính ứng dụng cao. Thông qua các đồ án chuyên ngành đa dạng và phong phú, sinh viên được học từ lý thuyết cơ bản đến khi có thể tự chuyển đổi ý tưởng thành sản phẩm hoàn thiện để trình diễn trước công chúng. Nhà trường còn tăng cường đầu tư phòng thực hành với không gian sáng tạo và các trang thiết bị chuyên dụng liên tục cập nhật, giúp sinh viên tiếp cận nhanh chóng với công nghệ và các xu hướng mới của ngành thời trang.\n\nĐiểm nổi bật của chương trình học là được xây dựng theo hướng kết nối với doanh nghiệp. Nhà trường thường xuyên kết hợp với các công ty trong và ngoài nước thuộc lĩnh vực thời trang để tổ chức các buổi tham quan, kiến tập, các cuộc thi thiết kế 